<a href="https://colab.research.google.com/github/richeung1/Fake-News-Detector/blob/Using-Transformers/Real_vs_Fake_News_Detector_with_DistilBERT_7_28_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch

In [12]:
# 40k rows dataset
#df2 = pd.read_csv('/content/drive/MyDrive/Machine Learning Colab Notebooks/Fake vs Real News/Real_Fake_News.csv')

In [3]:
# 2k rows dataset
df = pd.read_csv('/content/drive/MyDrive/Machine Learning Colab Notebooks/Fake vs Real News/Fake vs Real News First Trial/news_articles.csv')

In [4]:
df.head(5)

,author,published,title,text,language,site_url,main_img_url,type,label,title_without_stopwords,text_without_stopwords,hasImage
0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,muslims busted they stole millions in govt ben...,print they should pay all the back all the mon...,english,100percentfedup.com,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,bias,Real,muslims busted stole millions govt benefits,print pay back money plus interest entire fami...,1.0
1,reasoning with facts,2016-10-29T08:47:11.259+03:00,re why did attorney general loretta lynch plea...,why did attorney general loretta lynch plead t...,english,100percentfedup.com,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,bias,Real,attorney general loretta lynch plead fifth,attorney general loretta lynch plead fifth bar...,1.0
2,Barracuda Brigade,2016-10-31T01:41:49.479+02:00,breaking weiner cooperating with fbi on hillar...,red state \nfox news sunday reported this mor...,english,100percentfedup.com,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,bias,Real,breaking weiner cooperating fbi hillary email ...,red state fox news sunday reported morning ant...,1.0
3,Fed Up,2016-11-01T05:22:00.000+02:00,pin drop speech by father of daughter kidnappe...,email kayla mueller was a prisoner and torture...,english,100percentfedup.com,http://100percentfedup.com/wp-content/uploads/...,bias,Real,pin drop speech father daughter kidnapped kill...,email kayla mueller prisoner tortured isis cha...,1.0
4,Fed Up,2016-11-01T21:56:00.000+02:00,fantastic trumps point plan to reform healthc...,email healthcare reform to make america great ...,english,100percentfedup.com,http://100percentfedup.com/wp-content/uploads/...,bias,Real,fantastic trumps point plan reform healthcare ...,email healthcare reform make america great sin...,1.0


In [5]:
new_df = pd.DataFrame(df['text_without_stopwords'])

In [6]:
new_df['Real'] = df['label']
new_df = new_df.dropna()

In [7]:
new_df['Real'] = new_df['Real'].map({'Real': 1, 'Fake': 0})

In [8]:
new_df.head(5)
#df2.head(5)

,text_without_stopwords,Real
0,print pay back money plus interest entire fami...,1
1,attorney general loretta lynch plead fifth bar...,1
2,red state fox news sunday reported morning ant...,1
3,email kayla mueller prisoner tortured isis cha...,1
4,email healthcare reform make america great sin...,1


In [9]:
X = list(new_df.text_without_stopwords)
y = list(new_df.Real)

# X = list(df2.text)
# y = list(df2.label)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [11]:
!pip install transformers
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 43.5 MB/s 
     |████████████████████████████████| 596 kB 60.8 MB/s 
     |████████████████████████████████| 101 kB 12.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [13]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased", num_labels=2)

In [14]:
inputs = tokenizer.batch_encode_plus(
    X_train,
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding=True, 
    max_length=512,
    truncation=True,
    return_tensors='pt'
)

In [15]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler


input_ids = inputs['input_ids']
attn_mask = inputs['attention_mask']
labels = torch.tensor(y_train)

dataset_train = TensorDataset(input_ids, attn_mask, labels)
dataset_train

In [16]:
inputs_test = tokenizer.batch_encode_plus(
    X_test[:-300],
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding=True, 
    max_length=512,
    truncation=True,
    return_tensors='pt'
)

In [17]:
input_ids_test = inputs_test['input_ids']
attn_mask_test = inputs_test['attention_mask']
labels_test = torch.tensor(y_test[:-300])

dataset_test = TensorDataset(input_ids_test, attn_mask_test, labels_test)
dataset_test

In [18]:
X_val = X_test[300:]
y_val = y_test[300:]

In [19]:
inputs_val = tokenizer.batch_encode_plus(
    X_val,
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding=True, 
    max_length=512,
    truncation=True,
    return_tensors='pt'
)

In [20]:
input_ids_val = inputs_val['input_ids']
attn_mask_val = inputs_val['attention_mask']
labels_val = torch.tensor(y_val)

dataset_val = TensorDataset(input_ids_val, attn_mask_val, labels_val)
dataset_val

In [21]:
batchsize=16

dataloader_train = DataLoader(dataset_train, sampler=RandomSampler(dataset_train), batch_size=batchsize)

dataloader_test = DataLoader(dataset_test, sampler=SequentialSampler(dataset_test), batch_size=batchsize)

dataloader_val = DataLoader(dataset_val, sampler=SequentialSampler(dataset_val), batch_size=batchsize)

In [22]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
#model.load_state_dict(torch.load('/content/fakenews.pt'))
#model

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [23]:
from transformers import AdamW, get_linear_schedule_with_warmup

# optimizer = AdamW(model.parameters(), lr=1e-5, eps=1e-8, weight_decay=0.01)
# optimizer = AdamW(model.parameters(), lr=3e-5, eps=1e-8, weight_decay=0.01)
optimizer = AdamW(model.parameters(), lr=5e-5, eps=1e-8, weight_decay=0.01)
epochs=5
total_steps = len(dataloader_train) * epochs
warmup_steps = round(total_steps * 0.05)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [24]:
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef
from sklearn.metrics import f1_score, hamming_loss, classification_report
import numpy as np
'''f1 score'''
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')


'''Per-class accuracy'''
def accuracy_per_class(preds, labels):
    
    label_dict = {
        'fake': 0,
        'real': 1
    }
    
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [25]:
import random
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


'''Evaluation loop'''
def evaluate(dataloader):

    # Pass your defaults to wandb.init

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval().to(device)
    
    loss_val_total = 0
    predictions, true_vals = [], []

    
    for batch in dataloader:
        logits = 0
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }
        
        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()
        

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
        
    loss_val_avg = loss_val_total/len(dataloader) 
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
    return loss_val_avg, predictions, true_vals
    
    
def train():

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(device)
    
    for epoch in tqdm(range(1, epochs+1)):
    
        model.train().to(device)

        loss_train_total = 0

        progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
        for batch in progress_bar:

            model.zero_grad()
        
            batch = tuple(b.to(device) for b in batch)
        
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'labels':         batch[2],
                     }       

            outputs = model(**inputs)
          
            loss = outputs[0]
            loss_train_total += loss.item()
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            optimizer.step()
            scheduler.step()
        
            progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
        
        torch.save(model.state_dict(), f'./fakenews_2k.pt')
        
        tqdm.write(f'\nEpoch {epoch}')
    
        loss_train_avg = loss_train_total/len(dataloader_train)   
        tqdm.write(f'Training loss: {loss_train_avg}')
        
    
        val_loss, predictions, true_vals = evaluate(dataloader_test)
        val_f1 = f1_score_func(predictions, true_vals)
        accuracy_per_class(predictions, true_vals)
        tqdm.write(f'Validation loss: {val_loss}')
        tqdm.write(f'F1 Score (Weighted): {val_f1}')
        tqdm.write(f'train loss avg: {loss_train_avg}')

In [26]:
# train/eval on the 2k dataset
train()

cuda


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/90 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.6463023652633031
Class: fake
Accuracy: 201/208

Class: real
Accuracy: 22/106

Validation loss: 0.5577217489480972
F1 Score (Weighted): 0.6501740096189781
train loss avg: 0.6463023652633031


Epoch 2:   0%|          | 0/90 [00:00<?, ?it/s]

...


Epoch 3
Training loss: 0.23787659613622558
Class: fake
Accuracy: 188/208

Class: real
Accuracy: 75/106

Validation loss: 0.45831144154071807
F1 Score (Weighted): 0.8352273448313555
train loss avg: 0.23787659613622558


Epoch 4:   0%|          | 0/90 [00:00<?, ?it/s]


Epoch 4
Training loss: 0.12176706894404359
Class: fake
Accuracy: 193/208

Class: real
Accuracy: 69/106

Validation loss: 0.6507214950397611
F1 Score (Weighted): 0.8289662731123245
train loss avg: 0.12176706894404359


Epoch 5:   0%|          | 0/90 [00:00<?, ?it/s]


Epoch 5
Training loss: 0.06337689256761223
Class: fake
Accuracy: 184/208

Class: real
Accuracy: 79/106

Validation loss: 0.6623937705531716
F1 Score (Weighted): 0.8369953785270651
train loss avg: 0.06337689256761223


In [ ]:
# Use the 40k data set to continue scaling up, Train/eval on the 40k dataset

In [ ]:
# train()
# Load base model again and then load the new model weights. Play around with hyperparameters like optimizer and learning rate.
# because currently the train on the 40k is not learning anything. it is stuck on the same training loss and thereofore same F1 score.

In [27]:
val_loss, predictions, true_vals = evaluate(dataloader_val)

In [28]:
accuracy_per_class(predictions, true_vals)

Class: fake
Accuracy: 155/189

Class: real
Accuracy: 102/125



In [ ]:
# AdamW lr 1e-5 = 73.5 
            3e-5 = 79.3
            5e-5 = 77